In [1]:
# Check PyTorch version installed on this system
!python -c "import torch; print(torch.__version__)"

1.13.0+cu116


In [2]:
!nvidia-smi

Thu Dec 15 22:56:51 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   68C    P0    31W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
# Download the required modules
"""
Assign to TORCH with what you get from the cell above, E.g., export TORCH=1.13.0+cu116
"""
%env TORCH=1.13.0+cu116
!pip install torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install torch-geometric
!pip install torchmetrics
!pip install matplotlib
!pip install networkx
!pip install scipy

env: TORCH=1.13.0+cu116
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-1.13.0+cu116.html
     |████████████████████████████████| 9.4 MB 9.5 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-1.13.0+cu116.html
     |████████████████████████████████| 4.6 MB 9.5 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 564 kB 7.6 MB/s 
     |████████████████████████████████| 280 kB 75.3 MB/s 
  Created wheel for torch-geometric: filename=torch_geometric-2.2.0-py3-none-any.whl size=773302 sha256=42e53149bbf90ab77b035b125179d4549691d0ee065f1d557842a9e87e0757d0
  Stored in directory: /root/.cache/pip/wheels/59/a3/20/198928106d3169865ae73afcbd3d3d1796cf6b429b55c65378
Successfully built torch-geometric
  Att

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 512 kB 5.9 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
github_username="deeplearningtester"
github_repository="grl"
github_token = ""

%env github_repository={github_repository}

env: github_repository=grl


In [5]:
!git clone https://{github_token}@github.com/{github_username}/{github_repository}.git

Cloning into 'grl'...
remote: Enumerating objects: 78, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 78 (delta 1), reused 4 (delta 1), pack-reused 70
Unpacking objects: 100% (78/78), done.


In [6]:
%cd $github_repository

/content/grl


In [7]:
import torch
import random

random.seed(42)
torch.manual_seed(42)

In [8]:
from torch_geometric.datasets import PPI
from torch_geometric.loader import DataLoader
from torch_geometric.transforms import AddSelfLoops
import dataset

train_dataset = PPI(root='', split='train', transform=AddSelfLoops())
val_dataset = PPI(root='', split='val', transform=AddSelfLoops())
test_dataset = PPI(root='', split='test', transform=AddSelfLoops())

dataset.num_features = 50
dataset.num_labels = 121
dataset.train_loader = DataLoader(train_dataset, batch_size=2)
dataset.val_loader = DataLoader(val_dataset, batch_size=2)
dataset.test_loader = DataLoader(test_dataset, batch_size=2)

Extracting ./ppi.zip
Processing...
Done!


In [9]:
from evaluation import evaluate
from training_loop import train

In [10]:
from models import GATv1PPI
model = GATv1PPI(dataset.num_features, dataset.num_labels)
evaluate(model, dataset.test_loader)

(tensor(0.7045, device='cuda:0'), tensor(0.3902, device='cuda:0'))

In [11]:
ppi_train_params = {
  "lr": 5e-3,
  "weight_decay": 0,
  "epochs": 400,
  "patience": 100,
  "model_name": model.model_name
}

In [12]:
best_model, _, _, _, _, model_saver = train(model, ppi_train_params)
best_model = model
model_saver.download_best_model_state()

training model GATv1PPI
GATv1PPI(
  (layers): ModuleList(
    (0): GATLayer()
    (1): GATLayer()
    (2): GATLayer(
      (act): Identity()
    )
  )
)
training...
epoch 00000
	train_loss: 0.7525 | train_micro_f1: 0.4247
	val_loss: 0.5892 | val_micro_f1: 0.4316
epoch 00001
	train_loss: 0.5561 | train_micro_f1: 0.4739
	val_loss: 0.5164 | val_micro_f1: 0.5011
epoch 00002
	train_loss: 0.5177 | train_micro_f1: 0.4984
	val_loss: 0.5001 | val_micro_f1: 0.4951
epoch 00003
	train_loss: 0.5026 | train_micro_f1: 0.5265
	val_loss: 0.4895 | val_micro_f1: 0.5013
epoch 00004
	train_loss: 0.4922 | train_micro_f1: 0.5431
	val_loss: 0.4807 | val_micro_f1: 0.5431
epoch 00005
	train_loss: 0.4877 | train_micro_f1: 0.5571
	val_loss: 0.4823 | val_micro_f1: 0.4838
epoch 00006
	train_loss: 0.4774 | train_micro_f1: 0.5687
	val_loss: 0.4703 | val_micro_f1: 0.5152
epoch 00007
	train_loss: 0.4663 | train_micro_f1: 0.5869
	val_loss: 0.4658 | val_micro_f1: 0.5208
epoch 00008
	train_loss: 0.4552 | train_micro_f1: 0

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [13]:
from torch_geometric.utils import to_networkx

[test_graph] = list(dataset.test_loader)
test_graph_nx = to_networkx(test_graph)

print(test_graph)
print(test_graph_nx)

DataBatch(x=[5524, 50], edge_index=[2, 167500], y=[5524, 121], batch=[5524], ptr=[3])
DiGraph with 5524 nodes and 167500 edges
